# Google Colaboratoryによるクラウド学習

---



## Googleドライブへのアクセス権限を与える

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# 準備したデータセットのパス
LOAD_DATA_PATH = "/content/drive/MyDrive/workspace_jetracer/dataset/train/FaBoCircuit/0707003/camera/run202307071221fps5slow30lap/"

In [ ]:
# パスの確認
!ls $LOAD_DATA_PATH

In [ ]:
# データを確認
!ls $LOAD_DATA_PATH/xy

# JetRacerのPythonコードをダウンロード

In [ ]:
!wget https://raw.githubusercontent.com/NVIDIA-AI-IOT/jetracer/master/notebooks/utils.py
!wget https://raw.githubusercontent.com/NVIDIA-AI-IOT/jetracer/master/notebooks/xy_dataset.py

# トレーニング

In [ ]:
#pytorchを使用します。
import torch
import torchvision
import time
from xy_dataset import XYDataset
import torchvision.transforms as transforms


In [ ]:
# 属性は２つ
LOAD_CATEGORIES = ['xy','speed']
SAVE_CATEGORIES = ['xy','speed']
# 画像の前処理
TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# クラウドのGPUを使用する。
device = torch.device('cuda')

output_dim = 2*len(LOAD_CATEGORIES)

# １８層の畳み込み事前学習済みニューラルネットワークを使用
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)
model = model.to(device)


In [ ]:
BATCH_SIZE = 8
EPOCH = 120

optimizer = torch.optim.Adam(model.parameters())

def train_eval(is_training):
    global model
    dataset = XYDataset(LOAD_DATA_PATH, SAVE_CATEGORIES, TRANSFORMS, random_hflip=True)

    epoch_count = 1

    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()

        print("学習の開始")
        total_time = time.time()

        while epoch_count < EPOCH:
            start_time = time.time()

            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, category_idx, xy in iter(train_loader):
                # send data to device
                images = images.to(device)
                xy = xy.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute MSE loss over x, y coordinates for associated categories
                loss = 0.0
                for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                    loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
                loss /= len(category_idx)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                count = len(category_idx.flatten())
                i += count
                sum_loss += float(loss)

            end_time_sec = time.time() - start_time
            total_time_sec = time.time() - total_time
            print("epoch数:{0:d}/{1:d}回, 経過時間:{2:f}秒, 1epochの経過時間:{3:f}秒".format(epoch_count, EPOCH, total_time_sec, end_time_sec))
            epoch_count += 1
    except e:
        pass

    model = model.eval()

In [ ]:
# 学習開始と評価を行います。
train_eval(True)

In [ ]:
# 学習したモデルを保存
SAVE_MODEL = "result202307061730.pth"
torch.save(model.state_dict(), SAVE_MODEL)

# 検証画像を作成

In [ ]:
model_name = "/content/drive/MyDrive/workspace_jetracer/dataset/train/FaBoCircuit/model/model0707r3.pth"
device = torch.device('cuda')
model = model.to(device)
model.load_state_dict(torch.load(model_name))
print(model_name + "のモデルを【読込】ました。")


In [ ]:
import os
import cv2
from utils import preprocess
import datetime
no = 0
def make_movie(path):
  global output_path, no
  no += 1
  output_path = "result_" + str(no) + ".mp4"
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  outfh = cv2.VideoWriter(output_path, fourcc, 10, (224, 224))

  IMG_WIDTH=224
  IMG_HEIGHT=224
  dataset_path = path + "/xy/"
  print(dataset_path)
  xy_filenames = os.listdir(dataset_path)
  xy_filenames = sorted(xy_filenames)
  print(str(len(xy_filenames)) + "枚の画像から映像を作成")
  for xy_name in xy_filenames:
    img = cv2.imread(dataset_path + xy_name)
    preprocessed = preprocess(img)
    output = model(preprocessed).detach().cpu().numpy().flatten()
    result_x = output[0]
    result_y = output[1]
    result_speed = output[3]
    result_x = int(IMG_WIDTH * (result_x / 2.0 + 0.5))
    result_y = int(IMG_HEIGHT * (result_y / 2.0 + 0.5))
    result_speed = int(IMG_HEIGHT * (result_speed / 2.0 + 0.5))
    img = cv2.circle(img, (int(result_x), int(result_y)), 8, (255, 0, 0), 3)

    # Speed
    result_speed = output[3]
    result_speed = int(IMG_WIDTH * (result_speed / 2.0 + 0.5))
    if result_speed > 224:
        result_speed = 244
    elif result_speed < 0:
        result_speed = 0
    img = cv2.line(img,(218,0),(218,224),(0,0,0),5)
    img = cv2.line(img,(219,224-result_speed),(219,224),(0,140,255),3)
    img = cv2.putText(img,"speed:"+str(result_speed),(160,215),cv2.FONT_HERSHEY_SIMPLEX,0.3,(255,255,255))

    outfh.write(img)
  outfh.release()

# 検証動画を再生する

In [1]:
from IPython.display import HTML
from base64 import b64encode

import os

def convert_movie():
  global output_path, compressed_path
  print("圧縮変換")
  compressed_path = "play_" + output_path
  os.system(f"ffmpeg -i {output_path} -vcodec libx264 {compressed_path}")

In [ ]:
# 走行中のデータセットのGoogle Driveのフォルダを指定
CAMERA_PATH = '/content/drive/MyDrive/workspace_jetracer/dataset/train/FaBoCircuit/0707003/camera/run202307071221fps5slow30lap'
make_movie(CAMERA_PATH)
convert_movie()

In [ ]:
mp4 = open(compressed_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=224 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)